## Import required packages

In [1]:
import os
import time
import numpy as np
import tensorflow as tf

## Load the dataset

In [2]:
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt",
)

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, "rb").read().decode(encoding="utf-8")

# length of text is the number of characters in it
print(f"Length of text: {len(text)} characters")

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")

65 unique characters


## Text Preprocessing

### Vectorize the text

In [6]:
example_texts = ["abcdefg", "xyz"]

chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode("utf-8"))

F
i
r
s
t
 
C
i
t
i


In [14]:
seq_length = 100
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)
for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [15]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [16]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text


split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [17]:
dataset = sequences.map(split_input_target)

In [18]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


## Create training batches

In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

In [20]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [21]:
model = MyModel(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units)

## Model Testing

In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)"
    )

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [23]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [25]:
sampled_indices

array([33, 56, 25, 41, 35, 13, 44, 10, 14, 18,  9, 24, 53, 53, 11, 37, 44,
       51,  6, 32, 41, 56, 15, 31, 29,  4, 16, 46, 58, 34, 14, 36, 27, 46,
        0, 17, 26, 33, 13, 22,  9, 23, 10, 35, 16, 30, 32, 43, 12, 32, 29,
       14, 35, 17, 39,  9, 51, 63, 58, 52, 55, 52, 20, 10,  7, 11, 63, 57,
        4,  7,  7, 65, 64, 33, 40, 24, 59, 51, 33,  8, 58, 63, 60,  4, 46,
       20,  6, 28, 58, 57, 57, 58, 62, 41, 26, 41, 13, 38, 46, 24])

In [26]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b', this it is that makes your lady mourn!\n\nSecond Servant:\nO, this is it that makes your servants dro'

Next Char Predictions:
 b"TqLbV?e3AE.Knn:Xel'SbqBRP$CgsUAWNg[UNK]DMT?I.J3VCQSd;SPAVDZ.lxsmpmG3,:xr$,,zyTaKtlT-sxu$gG'OsrrswbMb?YgK"


In [27]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [28]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189891, shape=(), dtype=float32)


In [29]:
tf.exp(example_batch_mean_loss).numpy()

66.01559

In [30]:
model.compile(optimizer="adam", loss=loss)

In [31]:
# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, save_weights_only=True
)

In [32]:
EPOCHS = 20

In [33]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 21s 68ms/step - loss: 2.6975
Epoch 2/20
172/172 [==============================] - 11s 54ms/step - loss: 1.9742
Epoch 3/20
172/172 [==============================] - 11s 55ms/step - loss: 1.6955
Epoch 4/20
172/172 [==============================] - 12s 56ms/step - loss: 1.5364
Epoch 5/20
172/172 [==============================] - 11s 56ms/step - loss: 1.4392
Epoch 6/20
172/172 [==============================] - 11s 56ms/step - loss: 1.3727
Epoch 7/20
172/172 [==============================] - 11s 56ms/step - loss: 1.3204
Epoch 8/20
172/172 [==============================] - 11s 57ms/step - loss: 1.2760
Epoch 9/20
172/172 [==============================] - 12s 58ms/step - loss: 1.2350
Epoch 10/20
172/172 [==============================] - 11s 57ms/step - loss: 1.1944
Epoch 11/20
172/172 [==============================] - 12s 58ms/step - loss: 1.1543
Epoch 12/20
172/172 [==============================] - 12s 58ms/step - loss: 1.1122
E

## Generate text

In [34]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [35]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [36]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO:
D should stay like to delivat;
I put thee. Held Jurizel in tune?

LADY GREY:
The shadow will attend me for a husband.
Conspiratry, what news few or ege?

CLARENCE:
O Doricles, they say amen? he was not going to safe
And once we scouts: I doubt not out of driar;
The other made me epiled from that truth abed,
Beguiled in this aim, which to take up all, against the time:
Under your lordship thou do art
Which ne'er come in:
And hark you that which shall give him, and
in holy nughing when they live: pray you, commandment:'
Practise smoots, protector, let me be in lawling time;
But I will spend us love our cause to see, and show
no strengthening of respect, whence should excelds,
Cursed be sworn quickle, with his knave.'

BENVOLIO:
Go, by my provost, hearimorty most sorrow,
And fall beloved, and, for the royal dight
In all resign'd his head. Now I cermas,
Their bratchers come to swearity, or it perceived
I have done me from Menenius;
Because I will ever service.
Yet am I trust, Juliet

In [37]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, "\n\n" + "_" * 80)

tf.Tensor(
[b"ROMEO:\nGentle nothing.\n\nMENENIUS:\nWhat doth ye well?\nWould yet you do be mature is in you\nTo school-master with the Clarence how thyself and women\nWhat I will prove his, perchance of York have to save\nThy widdy hours in hoops and proclaims,\nErpining the old pant that were but sake, and run,\nNor the found shall scarlet him, brathers and Jove,' you\nDribbet! O gravel in this viceous battle's death;\nAnd you, good grace, would be gonerated,\nAccording to the one.\n\nDUCHESS OF YORK:\nO blessed sail, and loves not such about their honsur.\nCome, talk not of perjury? But\nOur Richard holding hard: having grown some devour\nI prove a bold opportal, let my sweet soul\nIs goodment I love myself.\n\nLEONTES:\nFirst, ask your brother's pardon.\n\nISABELLA:\nAy, but thou urgurst your majesty you have.\n\nPETRUCHIO:\nBe king and drown; now what is your began?\n\nKING RICHARD III:\nEven he, my lord, before his peace is fear;\nAnd, then, they saw he were worn irpore mine art,

## Export the generator

In [ ]:
tf.saved_model.save(one_step_model, "one_step")
one_step_reloaded = tf.saved_model.load("one_step")

In [ ]:
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))